# Symbolic amplitude model

Intensity function for two-pseudoscalar system:

$$
I(\Omega,\Phi) = 2\kappa\sum_{k}\left(
  (1-P_{\gamma})\left|\sum_{l,m}[l]_{m;k}^{(-)}\Re[Z_{l}^{m}(\Omega,\Phi)]\right|^2
  +(1-P_{\gamma})\left|\sum_{l,m}[l]_{m;k}^{(+)}\Im[Z_{l}^{m}(\Omega,\Phi)]\right|^2
  +(1+P_{\gamma})\left|\sum_{l,m}[l]_{m;k}^{(+)}\Re[Z_{l}^{m}(\Omega,\Phi)]\right|^2
  +(1+P_{\gamma})\left|\sum_{l,m}[l]_{m;k}^{(-)}\Im[Z_{l}^{m}(\Omega,\Phi)]\right|^2
\right)
$$

where $Z_{l}^{m}(\Omega,\Phi)=Y_{l}^{m}(\Omega)e^{-i\Phi}$ is a phase-rotated spherical harmonic, $\Omega$ is the solid angle, $\Phi$ is the angle between the production and polarization planes,  $P_{\gamma}$ is the polarization magnitude, $[l]$ are the partial wave amplitudes, $m$ is the associated m-projection, $k$ refers to a spin flip ($k=1$) or non-flip ($k=0$) at the nucleon vertex, and $\kappa$ is an overall phase space factor.

## Main intensity definition

In [ ]:
from __future__ import annotations

import sympy as sp
from ampform.io import aslatex
from ampform.sympy import PoolSum
from IPython.display import Math
from sympy.functions.special.spherical_harmonics import Ynm

In [ ]:
from sympy.printing.precedence import PRECEDENCE
from sympy.printing.printer import Printer


def _print_Indexed_latex(self, printer, *args):
    base = printer._print(self.base)
    indices = ", ".join(map(printer._print, self.indices))
    return f"{base}_{{{indices}}}"


sp.Indexed.precedence = PRECEDENCE["Pow"] - 1
sp.Indexed._latex = _print_Indexed_latex
Printer._global_settings["gothic_re_im"] = True

In [ ]:
k, l, m = sp.symbols("k l m")
phi, theta, Phi = sp.symbols("phi theta Phi", real=True)
kappa = sp.Symbol("kappa")
Py = sp.Symbol("P_gamma")

In [ ]:
from ampform.sympy import (
    UnevaluatedExpression,
    create_expression,
    implement_doit_method,
)


@implement_doit_method
class Znm(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, n, m, theta, phi, Phi, **hints) -> Znm:
        return create_expression(cls, n, m, theta, phi, Phi, **hints)

    def evaluate(self) -> sp.Mul:
        n, m, theta, phi, Phi = self.args
        return Ynm(n, m, theta, phi) * sp.exp(-sp.I * Phi)

    def _latex(self, printer, *args):
        n, m, theta, phi, Phi = map(printer._print, self.args)
        return Rf"Z_{n}^{m}({theta}, {phi}, {Phi})"

In [ ]:
znm = Znm(l, m, theta, phi, Phi)
Math(aslatex({znm: znm.doit()}))

In [ ]:
assert sp.im(znm) != 0

In [ ]:
from qrules.quantum_numbers import arange


def create_range(__min, __max) -> tuple[sp.Rational, ...]:
    return tuple(sp.Rational(x) for x in arange(__min, __max + 1))


max_l = 0
k_values = [0]
l_range = [2]  # create_range(0, max_l)
m_range = [0]  # create_range(-max_l, max_l)
lp = sp.IndexedBase("[l]^{(+)}", complex=True)
lm = sp.IndexedBase("[l]^{(-)}", complex=True)

intensity_expr = (
    2
    * kappa
    * PoolSum(
        (1 - Py)
        * sp.Abs(PoolSum(lm[m, k] * sp.re(znm), (l, l_range), (m, m_range))) ** 2
        + (1 - Py)
        * sp.Abs(PoolSum(lp[m, k] * sp.im(znm), (l, l_range), (m, m_range))) ** 2
        + (1 + Py)
        * sp.Abs(PoolSum(lp[m, k] * sp.re(znm), (l, l_range), (m, m_range))) ** 2
        + (1 + Py)
        * sp.Abs(PoolSum(lm[m, k] * sp.im(znm), (l, l_range), (m, m_range))) ** 2,
        (k, k_values),
    )
)
intensity_expr

In [ ]:
intensity_expr.doit()

## Suggested parameter values

In [ ]:
lp_defaults = {lp[m, k]: 1 for k in k_values for m in m_range}
lm_defaults = {lm[m, k]: 1 for k in k_values for m in m_range}
parameter_defaults = {
    Py: 1,
    kappa: sp.pi,
    **lp_defaults,
    **lm_defaults,
}

In [ ]:
substituted_expr = intensity_expr.doit().xreplace(parameter_defaults)
substituted_expr

In [ ]:
substituted_expr.simplify()

## Phase space investigation

In [ ]:
intensity_expr.doit().free_symbols

In [ ]:
plotted_expr = intensity_expr.doit().xreplace({kappa: sp.pi})
arguments = (theta, phi, Phi, Py, lm[m_range[0], 0], lp[m_range[0], 0])
intensity_func = sp.lambdify(arguments, plotted_expr, "numpy")
intensity_func

In [ ]:
import numpy as np

resolution = 100
cos_theta_array, phi_array = np.meshgrid(
    np.linspace(-1, +1, num=resolution),
    np.linspace(0, 2 * np.pi, num=resolution),
)
theta_array = np.arccos(cos_theta_array)

In [ ]:
lm_val = 0  # -0.3 + 0.2j
lp_val = 0.4 - 0.1j
Py_val = 1
Phi_val = 0
intensity_array = intensity_func(
    theta_array,
    phi_array,
    Phi_val,
    Py_val,
    lm_val,
    lp_val,
)
intensity_array.shape

In [ ]:
%matplotlib widget
import ipywidgets as w
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(7, 4))
ax.set_xlabel(R"$\phi$")
ax.set_ylabel(R"$\cos\theta$")
MESH = None


sliders = dict(
    Phi_val=w.FloatSlider(
        description="Φ",
        min=0,
        max=np.pi,
        value=1,
        step=np.pi / 10,
    ),
    Py_val=w.FloatSlider(
        description="Pγ",
        min=0,
        max=1,
        value=1,
        step=0.1,
    ),
    lm00_mag=w.FloatSlider(
        description="mag",
        min=0,
        max=2,
        value=1,
    ),
    lm00_phi=w.FloatSlider(
        description="phase",
        min=0,
        max=2 * np.pi,
        step=np.pi / 10,
        value=0,
    ),
    lp00_mag=w.FloatSlider(
        description="mag",
        min=0,
        max=2,
        value=0,
    ),
    lp00_phi=w.FloatSlider(
        description="phase",
        min=0,
        max=2 * np.pi,
        step=np.pi / 10,
        value=0,
    ),
    complex_rendering=w.RadioButtons(
        options=["real", "imag"],
        description="Complex Rendering:",
    ),
)


def plot(
    *, Phi_val, Py_val, lm00_mag, lm00_phi, lp00_mag, lp00_phi, complex_rendering
):
    global MESH
    Z = intensity_func(
        theta_array,
        phi_array,
        Phi_val,
        Py_val,
        lm00_mag * np.exp(lm00_phi * 1j),
        lp00_mag * np.exp(lp00_phi * 1j),
    )

    if complex_rendering == "imag":
        Z = Z.imag
        ax.set_title("Im $T$")
    elif complex_rendering == "real":
        Z = Z.real
        ax.set_title("Re $T$")
    else:
        raise NotImplementedError

    if MESH is None:
        MESH = ax.pcolormesh(phi_array, cos_theta_array, Z, cmap=plt.cm.coolwarm)
    else:
        MESH.set_array(Z)
    fig.canvas.draw()


UI = w.VBox(
    [
        sliders["complex_rendering"],
        sliders["Phi_val"],
        sliders["Py_val"],
        w.HBox([w.Label("[l]⁻₀₀"), sliders["lp00_mag"], sliders["lp00_phi"]]),
        w.HBox([w.Label("[l]⁺₀₀"), sliders["lm00_mag"], sliders["lm00_phi"]]),
    ]
)
output = w.interactive_output(plot, controls=sliders)
w.VBox([UI, output])